In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inputdata/glove.6B.100d.txt
/kaggle/input/inputdata/Amazon Product Reviews.csv


# Loading data and preprocessing

In [2]:
data_path = '../input/inputdata/Amazon Product Reviews.csv'
glove_path = '../input/inputdata/glove.6B.100d.txt'

In [3]:
data = pd.read_csv(data_path, encoding = 'utf-8')
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
data = data.dropna().reset_index(drop = True)

In [5]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

"didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

"he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

"I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

"i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

"mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

"mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

"she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

"should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

"this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

"there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

"they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

"we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

"what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

"where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

"why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                       
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

"you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

"you're": "you are", "you've": "you have"}                       

In [6]:
import re
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text

In [7]:
data['Summary'] = data['Summary'].apply(lambda x : preprocess(x))
data['Text'] = data['Text'].apply(lambda x : preprocess(x))
print(data['Summary'][20], '\n', data['Text'][20])

always fresh 
 husband twizzlers addict .  bought many times amazon government employees living overseas cannot get country assigned to .  always fresh tasty packed well arrive timely manner . 


In [8]:
x = data['Text']
y = data['Summary']
print(x[50], y[50], sep = '\n')

oatmeal good .  mushy soft like it .  quaker oats way go . 
like


Torch requirements

In [9]:
import random
import string
from io import open
import unidecode

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

In [10]:
import spacy
from collections import Counter
from tqdm import tqdm_notebook

In [11]:
from spacy.lang.en import English
spacy_en = English()
input_words = Counter()
output_words = Counter()

In [12]:
inputs = []
outputs = []

In [13]:
for i in tqdm_notebook(range(len(x))):
    in_tokens = spacy_en(x[i])
    out_tokens = spacy_en(y[i])
    
    if len(in_tokens) == 0 or len(out_tokens) == 0:
        continue
    for token in in_tokens:
        input_words.update([token.text.lower()])
    inputs.append([token.text.lower() for token in in_tokens] + ['_EOS'])
    
    for token in out_tokens:
        output_words.update([token.text.lower()])
    outputs.append([token.text.lower() for token in out_tokens] + ['_EOS'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [14]:
input_words = ['_SOS', '_EOS', '_UNK'] + sorted(input_words, key = input_words.get, reverse = True)
input_w2i = {o : i for i, o in enumerate(input_words)}
input_i2w = {i : o for i, o in enumerate(input_words)}
output_words = ['_SOS', '_EOS', '_UNK'] + sorted(output_words, key = output_words.get, reverse = True)
output_w2i = {o : i for i, o in enumerate(output_words)}
output_i2w = {i : o for i, o in enumerate(output_words)}

In [15]:
for i in range(len(inputs)):
    in_sentence = inputs[i]
    out_sentence = outputs[i]
    inputs[i] = [input_w2i[word] for word in in_sentence]
    outputs[i] = [output_w2i[word] for word in out_sentence]

# Building the model

In [16]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers = 1, drop_prob = 0):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, dropout = drop_prob, batch_first = True)
        
    def forward(self, inputs, hidden):
        embedd = self.embed(inputs)
        output, hidden = self.lstm(embedd, hidden)
        return output, hidden
    
    def init_hidden(self, batch_size = 1):
        return (torch.zeros(self.n_layers, self.batch_size, self.hidden_size, device = device), 
                torch.zeros(self.n_layers, self.batch_size, self.hidden_size, device = device))

In [17]:
class BahdanauDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers = 1, drop_prob = 0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.drop_prob = drop_prob
        
        self.embed = nn.Embedding(output_size, hidden_size)
        
        self.fc_hidden = nn.Linear(self.hidden_size, self.hidden_size, bias = False)
        self.fc_encoder = nn.Linear(self.hidden_size, self.hidden_size, bias = False)
        self.weight = nn.Parameter(torch.FloatTensor(1, hidden_size))
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.drop_prob)
        self.lstm = nn.LSTM(self.hidden_size * 2, self.hidden_size, batch_first=True)
        self.classifier = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, inputs, hidden, encoder_outputs):
        encoder_outputs = encoder_outputs.squeeze()
        # Embed input words
        embedded = self.embed(inputs).view(1, -1)
        embedded = self.dropout(embedded)

        # Calculating Alignment Scores
        x = torch.tanh(self.fc_hidden(hidden[0])+self.fc_encoder(encoder_outputs))
        alignment_scores = x.bmm(self.weight.unsqueeze(2))  

        # Softmaxing alignment scores to get Attention weights
        attn_weights = F.softmax(alignment_scores.view(1, -1), dim = 1)

        # Multiplying the Attention weights with encoder outputs to get the context vector
        context_vector = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        # Concatenating context vector with embedded input word
        output = torch.cat((embedded, context_vector[0]), 1).unsqueeze(0)
        # Passing the concatenated vector as input to the LSTM cell
        output, hidden = self.lstm(output, hidden)
        # Passing the LSTM output through a Linear layer acting as a classifier
        output = F.log_softmax(self.classifier(output[0]), dim = 1)
        return output, hidden, attn_weights

In [18]:
MAX_LENGTH = 50

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [20]:
hidden_size = 256
encoder = EncoderLSTM(len(input_text.word2idx), hidden_size).to(device)
decoder = BahdanauDecoder(hidden_size, len(output_text.word2idx)).to(device)

NameError: name 'input_text' is not defined

In [21]:
lr = 2e-3
encoder_optimizer = optim.Adam(encoder.parameters(), lr = lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr = lr)

NameError: name 'encoder' is not defined

In [22]:
EPOCHS = 10
teacher_forcing_ratio = 0.5
encoder.train()
decoder.train()

for epoch in range(1, EPOCHS+1):
    avg_loss = 0
    for i, sentence in enumerate(x_tensor):
        loss = 0.
        h = encoder.init_hidden()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        inp = torch.tensor(sentence).unsqueeze(0).to(device)
        encoder_out, h = encoder(inp, h)
        
        decoder_input = torch.tensor([], device = )

SyntaxError: invalid syntax (<ipython-input-22-bb44c2be4bcc>, line 17)